In [ ]:

import numpy
from multiprocessing import Manager

class fastaReader():


    def __init__(self):
        self.path = "/content/multifasta (1).fasta"

        self.seqs = list()
        self.names = list()
        self.read()


    def read(self):
        f = open(self.path, "r")
        lines = f.readlines()
        f.close()
        seq = ""
        for line in lines:
            if line[0] == ">":
                self.names.append(line[1:].strip())
                if seq != "":
                    self.seqs.append(seq)
                seq = ""
            else:
                seq += line.strip()
        self.seqs.append(seq)

In [ ]:
!pip install blosum

In [ ]:
import blosum as bl

class evaluadorBlosum():

    def __init__(self):
        matrix = bl.BLOSUM(62)

        self.matrix = matrix

    def showMatrix(self):
        print(self.matrix)

    def getScore(self, A, B):
        #si alguno de los dos es un gap
        if A == "-" or B == "-":
            return -8
        score = self.matrix[A][B]
        return score


    pass

In [ ]:
import copy
import math
from multiprocessing import Manager, Pool, managers
from pickle import FALSE, TRUE
#from evaluadorBlosum import evaluadorBlosum
import numpy
#from fastaReader import fastaReader
import random
from copy import copy
import copy
import concurrent.futures


class bacteria():


    def __init__(self, numBacterias):
        # manager = Manager()
        manager = Manager()
        self.blosumScore = manager.list(range(numBacterias))
        self.tablaAtract = manager.list(range(numBacterias))
        self.tablaRepel = manager.list(range(numBacterias))
        self.tablaInteraction = manager.list(range(numBacterias))
        self.tablaFitness = manager.list(range(numBacterias))
        self.granListaPares = manager.list(range(numBacterias))
        self.NFE = manager.list(range(numBacterias))

    def resetListas(self, numBacterias):
        manager = Manager()
        self.blosumScore = manager.list(range(numBacterias))
        self.tablaAtract = manager.list(range(numBacterias))
        self.tablaRepel = manager.list(range(numBacterias))
        self.tablaInteraction = manager.list(range(numBacterias))
        self.tablaFitness = manager.list(range(numBacterias))
        self.granListaPares = manager.list(range(numBacterias))
        self.NFE = manager.list(range(numBacterias))



    def cuadra(self, numSec, poblacion):
        #ciclo para recorrer poblacion
        for i in range(len(poblacion)):
            #obtiene las secuencias de la bacteria
            bacterTmp = poblacion[i]
            # print("bacterTmp: ", bacterTmp)
            bacterTmp = list(bacterTmp)
            # print("bacterTmp: ", bacterTmp)
            bacterTmp = bacterTmp[:numSec]
            # obtiene el tama�o de la secuencia m�s larga
            maxLen = 0
            for j in range(numSec):
                if len(bacterTmp[j]) > maxLen:
                    maxLen = len(bacterTmp[j])
                    #rellena con gaps las secuencias m�s cortas
                    for t in range(numSec):
                        gap_count = maxLen - len(bacterTmp[t])
                        if gap_count > 0:
                            bacterTmp[t].extend(["-"] * gap_count)
                            #actualiza la poblacion
                            poblacion[i] = tuple(bacterTmp)









    """metodo que recorre la matriz y elimina las columnas con gaps en todos los elementos"""
    def limpiaColumnas(self):
        i = 0
        while i < len(self.matrix.seqs[0]):
            if self.gapColumn(i):
                self.deleteCulmn(i)
            else:
                i += 1



        """metodo para eliminar un elemento especifico en cada secuencia"""
    def deleteCulmn(self, pos):
        for i in range(len(self.matrix.seqs)):
            self.matrix.seqs[i] = self.matrix.seqs[i][:pos] + self.matrix.seqs[i][pos+1:]



    """metodo para saber si alguna columna de self.matrix tiene  gap en todos los elementos"""
    def gapColumn(self, col):
        for i in range(len(self.matrix.seqs)):
            if self.matrix.seqs[i][col] != "-":
                return False
        return True



    def tumbo(self, numSec, poblacion, numGaps):
        #inserta un gap en una posicion aleatoria de una secuencia aleatoria
        #recorre la poblacion
        for i in range(len(poblacion)):
            #obtiene las secuencias de la bacteria
            bacterTmp = poblacion[i]
            bacterTmp = list(bacterTmp)
            # bacterTmp = bacterTmp[:numSec]
            #ciclo para insertar gaps
            for j in range(numGaps):
                #selecciona secuencia
                seqnum = random.randint(0, len(bacterTmp)-1)
                #selecciona posicion
                pos = random.randint(0, len(bacterTmp[seqnum]))
                part1 = bacterTmp[seqnum][:pos]
                part2 = bacterTmp[seqnum][pos:]
                temp = part1 + ["-"] + part2
                bacterTmp[seqnum] = temp
                #actualiza la poblacion
                poblacion[i] = tuple(bacterTmp)



    def creaGranListaPares(self, poblacion):
        # granListaPares = list(range(len(poblacion)))
        #ciclo para recorrer poblacion
        for i in range(len(poblacion)):  #recorre poblacion
            pares = list()
            bacterTmp = poblacion[i]
            bacterTmp = list(bacterTmp)
            #ciclo para recorrer secuencias
            for j in range(len(bacterTmp)):     #recorre secuencias de bacteria
                column = self.getColumn(bacterTmp, j)
                pares = pares + self.obtener_pares_unicos(column)
            self.granListaPares[i] = pares
            # print("Bacteria: ", i, " Pares: ", pares)

        # return self.granListaPares



    def evaluaFila(self, fila, num):
        evaluador = evaluadorBlosum()
        score = 0
        for par in fila:
            score += evaluador.getScore(par[0], par[1])
        self.blosumScore[num] = score

    def evaluaBlosum(self):
        with Pool() as pool:
            args = [(copy.deepcopy(self.granListaPares[i]), i) for i in range(len(self.granListaPares))]
            pool.starmap(self.evaluaFila, args)


    def getColumn(self, bacterTmp, colNum):
        column = []
        #obtiene las secuencias de la bacteria
        # bacterTmp = poblacion[bactNum]
        # bacterTmp = list(bacterTmp)
        #obtiene el caracter de cada secuencia en la columna
        for i in range(len(bacterTmp)):
            column.append(bacterTmp[i][colNum])
        return column





    def obtener_pares_unicos(self, columna):
        pares_unicos = set()
        for i in range(len(columna)):
            for j in range(i+1, len(columna)):
                par = tuple(sorted([columna[i], columna[j]]))
                pares_unicos.add(par)
        return list(pares_unicos)

    #------------------------------------------------------------Atract y Repel lineal




    def compute_diff(self, args):
        indexBacteria, otherBlosumScore, self.blosumScore, d, w = args
        diff = (self.blosumScore[indexBacteria] - otherBlosumScore) ** 2.0
        self.NFE[indexBacteria] += 1
        return d * numpy.exp(w * diff)

    def compute_cell_interaction(self, indexBacteria, d, w, atracTrue):
        with Pool() as pool:
            args = [(indexBacteria, otherBlosumScore, self.blosumScore, d, w) for otherBlosumScore in self.blosumScore]
            results = pool.map(self.compute_diff, args)
            pool.close()  # Close the pool to prevent any more tasks from being submitted
            pool.join()   # Wait for the worker processes to exit

        total = sum(results)

        if atracTrue:
            self.tablaAtract[indexBacteria] = total
        else:
            self.tablaRepel[indexBacteria] = total



    def creaTablaAtract(self, poblacion, d, w):                   #lineal
        for indexBacteria in range(len(poblacion)):
            self.compute_cell_interaction(indexBacteria,d, w, TRUE)
            # print("invocando indexBacteria numero: ", indexBacteria)
        # print("tablaAtract: ", self.tablaAtract)

    def creaTablaRepel(self, poblacion, d, w):                   #lineal
        for indexBacteria in range(len(poblacion)):
            self.compute_cell_interaction(indexBacteria,d, w, FALSE)
            # print("invocando indexBacteria numero: ", indexBacteria)
        # print("tablaAtract: ", self.tablaAtract)

    def creaTablasAtractRepel(self, poblacion, dAttr, wAttr, dRepel, wRepel):
        #invoca ambos metodos en paralelo
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.submit(self.creaTablaAtract, poblacion, dAttr, wAttr)
            executor.submit(self.creaTablaRepel, poblacion, dRepel, wRepel)




            #-----------------------------------------------------------

    def creaTablaInteraction(self):
        #llena la tabla con la suma de atract y repel
        for i in range(len(self.tablaAtract)):
            self.tablaInteraction[i] = self.tablaAtract[i] + self.tablaRepel[i]

    def creaTablaFitness(self):
        #llena la tabla con la suma de interaction y blosumScore
        for i in range(len(self.tablaInteraction)):
            valorBlsm = self.blosumScore[i]
            valorInteract = self.tablaInteraction[i]
            #suma ambos valores
            valorFitness =  valorBlsm + valorInteract

            self.tablaFitness[i] = valorFitness

    def getNFE(self):
        return sum(self.NFE)


    def obtieneBest(self, globalNFE):
        bestIdx = 0
        for i in range(len(self.tablaFitness)):
            if self.tablaFitness[i] > self.tablaFitness[bestIdx]:
                bestIdx = i
        print("-------------------   Best: ", bestIdx, " Fitness: ", self.tablaFitness[bestIdx], "BlosumScore ",  self.blosumScore[bestIdx], "Interaction: ", self.tablaInteraction[bestIdx], "NFE: ", globalNFE)
        return bestIdx, self.tablaFitness[bestIdx]

    def replaceWorst(self, poblacion, best):
        worst = 0
        for i in range(len(self.tablaFitness)):
            if self.tablaFitness[i] < self.tablaFitness[worst]:
                worst = i
        # print("Worst: ", worst,  "Blosum ",self.blosumScore[worst], "Fitness: ", self.tablaFitness[worst], "BlosumScore: ", self.blosumScore[worst], "Atract: ", self.tablaAtract[worst], "Repel: ", self.tablaRepel[worst], "Interaction: ", self.tablaInteraction[worst])
        #reemplaza la bacteria peor por una copia de la mejor
        poblacion[worst] = copy.deepcopy(poblacion[best])



In [ ]:
from copy import copy
from multiprocessing import Manager
import time
#from bacteria import bacteria
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from fastaReader import fastaReader
import copy

def run_algorithm(numeroDeBacterias, iteraciones, tumbo, dAttr, wAttr, hRep, wRep):
    # Leer las secuencias y nombres desde el archivo FASTA
    secuencias = fastaReader().seqs
    names = fastaReader().names

    # Convertir las secuencias en listas de caracteres
    for i in range(len(secuencias)):
        secuencias[i] = list(secuencias[i])

    globalNFE = 0  # Número de evaluaciones de la función objetivo
    manager = Manager()
    numSec = len(secuencias)
    poblacion = manager.list(range(numeroDeBacterias))
    names = manager.list(names)
    NFE = manager.list(range(numeroDeBacterias))

    # Crear la población inicial de bacterias
    def poblacionInicial():
        for i in range(numeroDeBacterias):
            bacterium = []
            for j in range(numSec):
                bacterium.append(secuencias[j])
            poblacion[i] = list(bacterium)

    # Inicializar el operador bacterial
    operadorBacterial = bacteria(numeroDeBacterias)
    veryBest = [None, None, None]  # [índice, fitness, secuencias]
    start_time = time.time()

    # Ejecutar el algoritmo para el número de iteraciones especificado
    poblacionInicial()
    for it in range(iteraciones):
        operadorBacterial.tumbo(numSec, poblacion, tumbo)
        operadorBacterial.cuadra(numSec, poblacion)
        operadorBacterial.creaGranListaPares(poblacion)
        operadorBacterial.evaluaBlosum()
        operadorBacterial.creaTablasAtractRepel(poblacion, dAttr, wAttr, hRep, wRep)
        operadorBacterial.creaTablaInteraction()
        operadorBacterial.creaTablaFitness()
        globalNFE += operadorBacterial.getNFE()
        bestIdx, bestFitness = operadorBacterial.obtieneBest(globalNFE)

        # Actualizar la mejor solución encontrada
        if (veryBest[0] is None) or (bestFitness > veryBest[1]):
            veryBest[0] = bestIdx
            veryBest[1] = bestFitness
            veryBest[2] = copy.deepcopy(poblacion[bestIdx])

        # Reemplazar la peor bacteria por una copia de la mejor
        operadorBacterial.replaceWorst(poblacion, veryBest[0])
        operadorBacterial.resetListas(numeroDeBacterias)

    # Calcular el tiempo de ejecución
    execution_time = time.time() - start_time

    # Devolver los resultados de la ejecución
    return {
        "fitness": veryBest[1],
        "time": execution_time,
        "interaction": operadorBacterial.tablaInteraction[veryBest[0]],
        "blosum_score": operadorBacterial.blosumScore[veryBest[0]]
    }

def performance_analysis(runs=30):
    results = []
    parameters = {
        "numeroDeBacterias": 15,
        "iteraciones": 10,
        "tumbo": 70,
        "dAttr": 0.06,
        "wAttr": 0.005,
        "hRep": 0.1,
        "wRep": 0.001
    }

    # Ejecutar las corridas
    for run in range(runs):
        print(f"Running iteration {run + 1}/{runs}")
        result = run_algorithm(**parameters)
        results.append(result)

    # Guardar los resultados en un DataFrame
    df = pd.DataFrame(results)
    df.to_csv("performance_results.csv", index=False)
    return df

def plot_results(df):
    # Crear gráficos para visualizar los resultados
    plt.figure(figsize=(12, 8))

    # Gráfico 1: Fitness vs Tiempo de ejecución
    plt.subplot(2, 2, 1)
    plt.scatter(df['time'], df['fitness'])
    plt.xlabel('Execution Time (s)')
    plt.ylabel('Fitness')
    plt.title('Fitness vs Execution Time')

    # Gráfico 2: Fitness vs Interacción
    plt.subplot(2, 2, 2)
    plt.scatter(df['interaction'], df['fitness'])
    plt.xlabel('Interaction')
    plt.ylabel('Fitness')
    plt.title('Fitness vs Interaction')

    # Gráfico 3: Fitness vs BLOSUM Score
    plt.subplot(2, 2, 3)
    plt.scatter(df['blosum_score'], df['fitness'])
    plt.xlabel('BLOSUM Score')
    plt.ylabel('Fitness')
    plt.title('Fitness vs BLOSUM Score')

    # Gráfico 4: Distribución de Fitness
    plt.subplot(2, 2, 4)
    plt.hist(df['fitness'], bins=20)
    plt.xlabel('Fitness')
    plt.ylabel('Frequency')
    plt.title('Fitness Distribution')

    # Ajustar el layout y guardar los gráficos
    plt.tight_layout()
    plt.savefig("performance_analysis.png")
    plt.show()

if __name__ == "__main__":
    # Ejecutar el análisis de desempeño con 30 corridas
    df = performance_analysis(runs=30)

    # Generar y mostrar los gráficos
    plot_results(df)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el archivo CSV
df = pd.read_csv('/content/performance_results.csv')

# Calcular la matriz de correlación
correlation_matrix = df.corr()

# Crear el heatmap de correlación
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.3f', linewidths=0.5, square=True)

# Configurar el título
plt.title('Mapa de Correlación de Variables')
plt.show()